In [1]:
# Imports

import pandas as pd
import json

from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()
# Extract path variables
cdc_tickets_path = os.getenv('CDC_TICKETS_PATH')
mcmn_tickets_path = os.getenv('MCMN_TICKETS_PATH')

In [2]:
def json_list_to_dataframe(json_list):
    rows = []

    for item in json_list:
        number = item.get("Number")
        resolution = item.get("resolution_note")

        message_blocks = []
        for msg in item.get("messages", []):
            timestamp = msg.get("timestamp") or "NO_TIMESTAMP"
            text = msg.get("text", "").strip()

            message_blocks.append(
                f"[{timestamp}]\n{text}"
            )

        timestamped_text_exchange = "\n\n---\n\n".join(message_blocks)

        rows.append({
            "number": number,
            "timestamped_text_exchange": timestamped_text_exchange,
            "resolution": resolution
        })

    return pd.DataFrame(rows)

In [3]:
# Load MCMN tickets from JSON file
with open(mcmn_tickets_path, 'r', encoding='utf-8') as f:
    mcmn_data = json.load(f)

# Flatten JSON data for MCMN tickets
flat_data = []
for ticket in mcmn_data:
    for message in ticket['messages']:
        flat_data.append({
            'Number': ticket['Number'],
            'timestamp': message['timestamp'],
            'text': message['text'],
            'resolution_note': ticket['resolution_note']
        })

# Create and preprocess DataFrame from flattened MCMN data
df_mcmn = pd.DataFrame(flat_data)
df_mcmn['timestamp'] = pd.to_datetime(df_mcmn['timestamp'], errors='coerce')
df_mcmn = df_mcmn.dropna(subset=['timestamp'])

In [4]:
df = json_list_to_dataframe([mcmn_data[0]])


In [5]:
print(df['timestamped_text_exchange'][0])

[NO_TIMESTAMP]
error for material# 10456231
Vendor Material No. does not match with Siemens print no. Please correct accordingly.

---

[2025-04-30T09:24:01]
error for material# 10456231
Vendor Material No. does not match with Siemens print no. Please correct accordingly.

---

[2025-04-30T09:31:21]
Hello Hassan,

Can we have a short call to discuss on the incident.

Regards,
PMD Team

---

[2025-05-05T09:50:17]
Hello Hassan,

Can we have a short call to discuss on the incident.

Regards,
PMD Team

---

[2025-05-05T13:50:03]
reply from: hassan.ibrahim@siemens-healthineers.com

Can you please check my calendar and set up a meeting accordingly?

---

[2025-05-06T08:11:46]
Hello Hassan,

I have shared you the meeting request.

Regards,
PMD Team


In [7]:
mcmn_data[0]

{'Number': 'INC4964672',
 'messages': [{'timestamp': None,
   'text': 'error for material# 10456231\nVendor Material No. does not match with Siemens print no. Please correct accordingly.'},
  {'timestamp': '2025-04-30T09:24:01',
   'text': 'error for material# 10456231\nVendor Material No. does not match with Siemens print no. Please correct accordingly.'},
  {'timestamp': '2025-04-30T09:31:21',
   'text': 'Hello Hassan,\n\nCan we have a short call to discuss on the incident.\n\nRegards,\nPMD Team'},
  {'timestamp': '2025-05-05T09:50:17',
   'text': 'Hello Hassan,\n\nCan we have a short call to discuss on the incident.\n\nRegards,\nPMD Team'},
  {'timestamp': '2025-05-05T13:50:03',
   'text': 'reply from: hassan.ibrahim@siemens-healthineers.com\n\nCan you please check my calendar and set up a meeting accordingly?'},
  {'timestamp': '2025-05-06T08:11:46',
   'text': 'Hello Hassan,\n\nI have shared you the meeting request.\n\nRegards,\nPMD Team'}],
 'resolution_note': None}